In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import tqdm
import sklearn, sklearn.metrics

In [3]:
import torchxrayvision as xrv

In [4]:
print(xrv.datasets.NIH_Google_Dataset.__doc__)


    Chest Radiograph Interpretation with Deep Learning Models: Assessment with 
    Radiologist-adjudicated Reference Standards and Population-adjusted Evaluation
    Anna Majkowska, Sid Mittal, David F. Steiner, Joshua J. Reicher, Scott Mayer 
    McKinney, Gavin E. Duggan, Krish Eswaran, Po-Hsuan Cameron Chen, Yun Liu, 
    Sreenivasa Raju Kalidindi, Alexander Ding, Greg S. Corrado, Daniel Tse, and 
    Shravya Shetty. Radiology 2020
    
    https://pubs.rsna.org/doi/10.1148/radiol.2019191293
    


In [5]:
# Use XRV transforms to crop and resize the images
transforms = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                             xrv.datasets.XRayResizer(224)])

# Load Google dataset and PyTorch dataloader
dataset = xrv.datasets.NIH_Google_Dataset(imgpath="/Users/ieee8023/Datasets/NIH/images-224",
                                          transform=transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

# Load pre-trained model and erase classifier
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.op_threshs = None # prevent pre-trained model calibration
model.classifier = torch.nn.Linear(1024,1) # reinitialize classifier

optimizer = torch.optim.Adam(model.classifier.parameters()) # only train classifier
criterion = torch.nn.BCEWithLogitsLoss()



In [6]:
# training loop (can run on cpu)
for i, batch in enumerate(dataloader):
    if i > 20: break
    outputs = model(batch["img"])
    targets = batch["lab"][:, dataset.pathologies.index("Lung Opacity"), None]
    loss = criterion(outputs, targets)
    print(i, loss.detach().cpu().numpy())
    loss.backward()
    optimizer.step()

0 0.7197195
1 0.7032325
2 0.7019479
3 0.6871945
4 0.7000455
5 0.66800505
6 0.6873612
7 0.6366669
8 0.5571896
9 0.59965414
10 0.5273169
11 0.50774366
12 0.5894309
13 0.6490113
14 0.6630047
15 0.46392265
16 0.43167824
17 0.6394826
18 0.6443106
19 0.5077342
20 0.57959396


In [128]:
sample = dataset[0]

In [129]:
out = model(torch.from_numpy(sample["img"]).unsqueeze(0))
out = torch.sigmoid(out)

In [130]:
out

tensor([[0.3993]], grad_fn=<SigmoidBackward>)

In [131]:
labels = []
preds = []
with torch.inference_mode():
    for i in range(20):
        sample = dataset[i]
        label = sample["lab"][dataset.pathologies.index("Lung Opacity")]
        labels.append(label)
        pred = model(torch.from_numpy(sample["img"]).unsqueeze(0))
        pred = torch.sigmoid(pred).detach().numpy()[0][0]
        preds.append(pred)
        print(label, pred)

0.0 0.399319
1.0 0.4504822
1.0 0.46033397
0.0 0.33570313
0.0 0.39914772
0.0 0.4095046
0.0 0.42468312
0.0 0.39061117
1.0 0.43495628
1.0 0.38757616
0.0 0.45992658
0.0 0.42239773
1.0 0.47049743
0.0 0.37503645
0.0 0.39930966
0.0 0.42051744
0.0 0.37670723
0.0 0.3471031
0.0 0.3745227
1.0 0.4336192


In [132]:
sklearn.metrics.roc_auc_score(labels, preds)

0.8571428571428572